# Clean CIGALE results
Here we join the runs from CIGALE of the three different redsfhit ranges that we use (Low, Intermediate and High). We also present the first figures of this work. First, we import the important packages

In [1]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from astropy.table import Table, join, vstack
from scipy.stats import binned_statistic

In [3]:
# User shortcuts
def Perc75(array):
    return(np.nanpercentile(array, 75))


def Perc25(array):
    return(np.nanpercentile(array, 25))

## Joining CIGALE files

We read the results from the models we are going to use 

In [4]:
# SKIRTOR models
ModelSL = Table.read('CIGALEOutputs/results_SKLz_3Dic.fits', format='fits')
ModelS0L = Table.read('CIGALEOutputs/results_SKLz0_3Dic.fits', format='fits')

ModelSI = Table.read('CIGALEOutputs/results_SKIz_3Dic.fits', format='fits')
ModelS0I = Table.read('CIGALEOutputs/results_SKIz0_3Dic.fits', format='fits')

ModelSH = Table.read('CIGALEOutputs/results_SKHz_3Dic.fits', format='fits')
ModelS0H = Table.read('CIGALEOutputs/results_SKHz0_3Dic.fits', format='fits')

# Fritz models
ModelFL = Table.read('CIGALEOutputs/results_FrLz_3Dic.fits', format='fits')
ModelFI = Table.read('CIGALEOutputs/results_FrIz_3Dic.fits', format='fits')
ModelFH = Table.read('CIGALEOutputs/results_FrHz_3Dic.fits', format='fits')

ModelF0 = Table.read('CIGALEOutputs/results_Fr0_3Dic.fits', format='fits')

# Model Two-angles
ModelS37 = Table.read('CIGALEOutputs/results_SK3070_3Dic.fits', format='fits')
ModelF37 = Table.read('CIGALEOutputs/results_Fr3070_3Dic.fits', format='fits')

Then, we stack the tables that where separated for simplicity, and reorganize the list of galaxies so we can compare them with the runs of all the galaxies. Keep in mind that Models_SK0 and Models_Fr0 are practically the same models with some numerical differences (We ran the two different models for consistency and tested that the results are the same).

In [5]:
Models_SK  = vstack([ModelSL, ModelSI, ModelSH])
Models_SK0 = vstack([ModelS0L, ModelS0I, ModelS0H])
Models_Fr  = vstack([ModelFL, ModelFI, ModelFH])
Models_Fr0 = ModelF0[np.concatenate([where(ModelF0['id'] == oid)[0] for oid in Models_Fr['id']])]
Models_S37 = ModelS37[np.concatenate([where(ModelS37['id'] == oid)[0] for oid in Models_SK['id']])]
Models_F37 = ModelF37[np.concatenate([where(ModelF37['id'] == oid)[0] for oid in Models_Fr['id']])]

## Checking $\chi_{\rm{red}}^{2}$ distributions 
Here we calculate the average (percentiles) values of the different $\chi_{\rm{red}}^{2}$ in the No-AGN and AGN models. 
First, with the Fritz models.

In [6]:
print('75%: ', Perc75(log10(Models_SK0['best.reduced_chi_square']/Models_SK['best.reduced_chi_square'])))
print('50%: ', nanmedian(log10(Models_SK0['best.reduced_chi_square']/Models_SK['best.reduced_chi_square'])))
print('25%: ', Perc25(log10(Models_SK0['best.reduced_chi_square']/Models_SK['best.reduced_chi_square'])))

75%:  0.9125003960829409
50%:  0.582587542930501
25%:  0.2152682679613906


Then, with the SKIRTOR models

In [7]:
print('75%: ', Perc75(log10(Models_Fr0['best.reduced_chi_square']/Models_Fr['best.reduced_chi_square'])))
print('50%: ', nanmedian(log10(Models_Fr0['best.reduced_chi_square']/Models_Fr['best.reduced_chi_square'])))
print('25%: ', Perc25(log10(Models_Fr0['best.reduced_chi_square']/Models_Fr['best.reduced_chi_square'])))

75%:  0.8667417063156352
50%:  0.5597975132555366
25%:  0.20742111197427698


##### FIGURE 2

We create an image between $\chi_{\rm{red}}^{2}$ and the $f_{\rm{AGN}}$ to present how important it is to include the AGN in the models. With this figure, we can asses for which galaxies is good to assume a no-AGN model ($f_{\rm{AGN}}=0$). We also present in a separate panel the  $\chi_{\rm{red}}^{2}$  distribution. 

We calculate the running medians of the trends between $\chi_{\rm{red}}^{2}$ and the $f_{\rm{AGN}}$.

In [8]:
bin_means, bin_edges, binnumber = binned_statistic(Models_SK['bayes.agn.fracAGN'],
                                                   Models_SK['best.reduced_chi_square'], statistic=nanmedian, bins=np.linspace(0.1, 0.9, 17))
bin_means2, bin_edges2, binnumber2 = binned_statistic(Models_SK['bayes.agn.fracAGN'],
                                                      Models_SK0['best.reduced_chi_square'], statistic=nanmedian, bins=np.linspace(0.1, 0.9, 17))
bin_means3, bin_edges3, binnumber3 = binned_statistic(Models_Fr['bayes.agn.fracAGN'],
                                                      Models_Fr['best.reduced_chi_square'], statistic=nanmedian, bins=np.linspace(0.1, 0.9, 17))
bin_means4, bin_edges4, binnumber4 = binned_statistic(Models_Fr['bayes.agn.fracAGN'],
                                                      Models_Fr0['best.reduced_chi_square'], statistic=nanmedian, bins=np.linspace(0.1, 0.9, 17))
binsChiS = np.linspace(-2.0, 2.5, 46)  # Assuming Chisquare bins in this ranges

Here the values for Fritz and SKIRTOR No-AGN models change due to a different fAGN in the models. 

Then, we plot the FIGURE 2

In [16]:
# with plt.style.context('seaborn-talk'):
with plt.style.context('seaborn-paper'):
    plt.rcParams.update({'font.size': 12})
    plt.rcParams.update({'axes.labelsize': 12})
    plt.rcParams.update({'legend.fontsize': 10})
    plt.rcParams.update({'ytick.labelsize': 10})
    plt.rcParams.update({'xtick.labelsize': 10})
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6.75, 9.44),
                                   sharex=True, gridspec_kw={'height_ratios': [1, 3]})
    ax2.scatter(log10(Models_SK['best.reduced_chi_square']),
                Models_SK['bayes.agn.fracAGN'], alpha=0.02)
    ax2.scatter(log10(Models_SK0['best.reduced_chi_square']),
                Models_SK['bayes.agn.fracAGN'], alpha=0.02)
    ax2.scatter(log10(Models_Fr['best.reduced_chi_square']),
                Models_Fr['bayes.agn.fracAGN'], alpha=0.02)
    ax2.scatter(log10(Models_Fr0['best.reduced_chi_square']),
                Models_Fr['bayes.agn.fracAGN'], alpha=0.02)
    ax2.vlines(log10(bin_means), bin_edges[:-1], bin_edges[1:], colors='C0', lw=3,
               label='SKIRTOR')
    ax2.vlines(log10(bin_means2), bin_edges2[:-1], bin_edges2[1:], colors='C1', lw=3,
               label='SKIRTOR No-AGN')
    ax2.vlines(log10(bin_means3), bin_edges3[:-1], bin_edges3[1:], colors='C2', lw=3,
               label='Fritz')
    ax2.vlines(log10(bin_means4), bin_edges4[:-1], bin_edges4[1:], colors='C3', lw=3,
               label='Fritz No-AGN')
    ax2.axvline(x=0.5, ls='--', color='gray')
    ax2.axvline(x=-0.5, ls='--', color='gray')
    ax2.set_xlabel(r'$\log(\chi^2_{\rm{red}})$')
    ax2.set_ylabel('AGN Fraction')
    ax2.legend(loc='best')
    ax1.set_xlabel('AGN Fraction')
    ax1.hist(log10(Models_SK['best.reduced_chi_square']),
             histtype='step', bins=binsChiS, label='SKIRTOR', lw=2)
    ax1.hist(log10(Models_SK0['best.reduced_chi_square']),
             histtype='step', bins=binsChiS, label='SKIRTOR No-AGN', lw=2)
    ax1.hist(log10(Models_Fr['best.reduced_chi_square']),
             histtype='step', bins=binsChiS, label='Fritz', lw=2)
    ax1.hist(log10(Models_Fr0['best.reduced_chi_square']),
             histtype='step', bins=binsChiS, label='Fritz No-AGN', lw=2)
    ax1.axvline(x=0.5, ls='--', color='gray')
    ax1.axvline(x=-0.5, ls='--', color='gray')
    ax1.set_ylabel('# Galaxies')
    ax1.set_xlim(-2, 2)
    ax2.set_ylim(0.08, 0.92)
    subplots_adjust(wspace=0, hspace=0)
    plt.savefig('Figures/F2.pdf',dpi=300,bbox_inches = 'tight') # For Paper
#     plt.savefig('Talk_Figures/F2.png',dpi=300,bbox_inches = 'tight') # For Presentation

## Cleaning
We remove all galaxies with:
- Bad fittings: $\chi_{\rm{red}}^{2}$ higher than $10^{0.5}$ (Underestimated) and lower than $10^{-0.5}$ (Overestimated).
- Galaxies where a no-AGN is probable
- Galaxies with a bad constrained SFR

We additionally print the median values for the simple Fritz and SKIRTOR models.

In [9]:
SClean = Models_SK[np.logical_and.reduce([Models_SK['best.reduced_chi_square'] <= 10**0.5,
                                          Models_SK['best.reduced_chi_square'] >= 10**-0.5,
                                          Models_SK['best.reduced_chi_square'] < Models_SK0['best.reduced_chi_square'],
                                          (Models_SK['bayes.sfh.sfr_err']/(Models_SK['bayes.sfh.sfr']*log(10))) <= 1])]
FClean = Models_Fr[np.logical_and.reduce([Models_Fr['best.reduced_chi_square'] <= 10**0.5,
                                          Models_Fr['best.reduced_chi_square'] >= 10**-0.5,
                                          Models_Fr['best.reduced_chi_square'] < Models_Fr0['best.reduced_chi_square'],
                                          (Models_Fr['bayes.sfh.sfr_err']/(Models_Fr['bayes.sfh.sfr']*log(10))) <= 1])]
SClean37 = Models_S37[np.logical_and.reduce([Models_S37['best.reduced_chi_square'] <= 10**0.5,
                                             Models_S37['best.reduced_chi_square'] >= 10**-0.5,
                                             Models_S37['best.reduced_chi_square'] < Models_SK0['best.reduced_chi_square'],
                                             (Models_S37['bayes.sfh.sfr_err']/(Models_S37['bayes.sfh.sfr']*log(10))) <= 1])]
FClean37 = Models_F37[np.logical_and.reduce([Models_F37['best.reduced_chi_square'] <= 10**0.5,
                                             Models_F37['best.reduced_chi_square'] >= 10**-0.5,
                                             Models_F37['best.reduced_chi_square'] < Models_Fr0['best.reduced_chi_square'],
                                             (Models_F37['bayes.sfh.sfr_err']/(Models_F37['bayes.sfh.sfr']*log(10))) <= 1])]

print(nanmedian(SClean['best.reduced_chi_square']),
      nanmedian(FClean['best.reduced_chi_square']))
print(Perc25(SClean['best.reduced_chi_square']),
      Perc25(FClean['best.reduced_chi_square']))
print(Perc75(SClean['best.reduced_chi_square']),
      Perc75(FClean['best.reduced_chi_square']))

0.9491982106048291 0.9814227689290372
0.5996758469826187 0.6266786960476942
1.5358495537730237 1.572620314503632


We rename the Fritz models as $\Psi=i-90$ and the thermal (dust) luminosity to compare with SKIRTOR model. 

In [10]:
FClean.rename_column('bayes.agn.psy', 'bayes.agn.i')
FClean.rename_column('bayes.agn.psy_err', 'bayes.agn.i_err')
FClean.rename_column('bayes.agn.therm_luminosity', 'bayes.agn.dust_luminosity')
FClean.rename_column('bayes.agn.therm_luminosity_err',
                     'bayes.agn.dust_luminosity_err')
FClean['bayes.agn.i'] = 90-FClean['bayes.agn.i']

FClean37.rename_column('bayes.agn.psy', 'bayes.agn.i')
FClean37.rename_column('bayes.agn.psy_err', 'bayes.agn.i_err')
FClean37.rename_column('bayes.agn.therm_luminosity',
                       'bayes.agn.dust_luminosity')
FClean37.rename_column('bayes.agn.therm_luminosity_err',
                       'bayes.agn.dust_luminosity_err')
FClean37['bayes.agn.i'] = 90-FClean37['bayes.agn.i']

Finally, we save the files of the cleaned results

In [ ]:
SClean.write('CIGALEOutputs/Cleanresults_SK.fits', format='fits')
FClean.write('CIGALEOutputs/Cleanresults_Fr.fits', format='fits')
SClean37.write('CIGALEOutputs/Cleanresults_SK37.fits', format='fits')
FClean37.write('CIGALEOutputs/Cleanresults_Fr37.fits', format='fits')

## Final numbers and counts by classification type
First, we present the final numbers of galaxies that are well-fitted to be analysed in this work

In [12]:
# For SK models
print('Total number of galaxies:', len(SClean))
Removed_Galaxies = len(Models_SK)-len(SClean)
Cond1 = sum(np.logical_or(Models_SK['best.reduced_chi_square'] >= 10**0.5,
                          Models_SK['best.reduced_chi_square'] <= 10**-0.5))/Removed_Galaxies
Cond2 = sum(Models_SK['best.reduced_chi_square'] >=
            Models_SK0['best.reduced_chi_square'])/Removed_Galaxies
Cond3 = sum((Models_SK['bayes.sfh.sfr_err'] /
             Models_SK['bayes.sfh.sfr']) > 3)/Removed_Galaxies
print('Percentage removed galaxies depending on condition:', Cond1, Cond2, Cond3)
Cond11 = sum(Models_SK['best.reduced_chi_square'] >= 10**0.5)/sum(np.logical_or(
    Models_SK['best.reduced_chi_square'] >= 10**0.5, Models_SK['best.reduced_chi_square'] <= 10**-0.5))
Cond12 = sum(Models_SK['best.reduced_chi_square'] <= 10**-0.5)/sum(np.logical_or(
    Models_SK['best.reduced_chi_square'] >= 10**0.5, Models_SK['best.reduced_chi_square'] <= 10**-0.5))
print('Percentage over and under fitting:', Cond11, Cond12)

Total number of galaxies: 8094
Percentage removed galaxies depending on condition: 0.7171797418073486 0.06792452830188679 0.26137040714995036
Percentage over and under fitting: 0.5131542509000276 0.4868457490999723


In [13]:
# For Fr models
print('Total number of galaxies:', len(FClean))
Removed_Galaxies = len(Models_Fr)-len(FClean)
Cond1 = sum(np.logical_or(Models_Fr['best.reduced_chi_square'] >= 10**0.5,
                          Models_Fr['best.reduced_chi_square'] <= 10**-0.5))/Removed_Galaxies
Cond2 = sum(Models_Fr['best.reduced_chi_square'] >=
            Models_Fr0['best.reduced_chi_square'])/Removed_Galaxies
Cond3 = sum((Models_Fr['bayes.sfh.sfr_err'] /
             Models_Fr['bayes.sfh.sfr']) > 3)/Removed_Galaxies
print('Percentage removed galaxies depending on condition:', Cond1, Cond2, Cond3)
Cond11 = sum(Models_Fr['best.reduced_chi_square'] >= 10**0.5)/sum(np.logical_or(
    Models_Fr['best.reduced_chi_square'] >= 10**0.5, Models_Fr['best.reduced_chi_square'] <= 10**-0.5))
Cond12 = sum(Models_Fr['best.reduced_chi_square'] <= 10**-0.5)/sum(np.logical_or(
    Models_Fr['best.reduced_chi_square'] >= 10**0.5, Models_Fr['best.reduced_chi_square'] <= 10**-0.5))
print('Percentage over and under fitting:', Cond11, Cond12)

Total number of galaxies: 8583
Percentage removed galaxies depending on condition: 0.7602287725472944 0.06577210734711834 0.23911130664320282
Percentage over and under fitting: 0.5607638888888888 0.4392361111111111


In [14]:
# For SK37 models
print('Total number of galaxies:', len(SClean37))
Removed_Galaxies = len(Models_S37)-len(SClean37)
Cond1 = sum(np.logical_or(Models_S37['best.reduced_chi_square'] >= 10**0.5,
                          Models_S37['best.reduced_chi_square'] <= 10**-0.5))/Removed_Galaxies
Cond2 = sum(Models_S37['best.reduced_chi_square'] >=
            Models_SK0['best.reduced_chi_square'])/Removed_Galaxies
Cond3 = sum((Models_S37['bayes.sfh.sfr_err'] /
             Models_S37['bayes.sfh.sfr']) > 3)/Removed_Galaxies
print('Percentage removed galaxies depending on condition:', Cond1, Cond2, Cond3)
Cond11 = sum(Models_S37['best.reduced_chi_square'] >= 10**0.5)/sum(np.logical_or(
    Models_S37['best.reduced_chi_square'] >= 10**0.5, Models_S37['best.reduced_chi_square'] <= 10**-0.5))
Cond12 = sum(Models_S37['best.reduced_chi_square'] <= 10**-0.5)/sum(np.logical_or(
    Models_S37['best.reduced_chi_square'] >= 10**0.5, Models_S37['best.reduced_chi_square'] <= 10**-0.5))
print('Percentage over and under fitting:', Cond11, Cond12)

Total number of galaxies: 8136
Percentage removed galaxies depending on condition: 0.7083917484478269 0.09653514920889245 0.25936310835169235
Percentage over and under fitting: 0.5877862595419847 0.4122137404580153


In [15]:
# For Fr37 models
print('Total number of galaxies:', len(FClean37))
Removed_Galaxies = len(Models_F37)-len(FClean37)
Cond1 = sum(np.logical_or(Models_F37['best.reduced_chi_square'] >= 10**0.5,
                          Models_F37['best.reduced_chi_square'] <= 10**-0.5))/Removed_Galaxies
Cond2 = sum(Models_F37['best.reduced_chi_square'] >=
            Models_Fr0['best.reduced_chi_square'])/Removed_Galaxies
Cond3 = sum((Models_F37['bayes.sfh.sfr_err'] /
             Models_F37['bayes.sfh.sfr']) > 3)/Removed_Galaxies
print('Percentage removed galaxies depending on condition:', Cond1, Cond2, Cond3)
Cond11 = sum(Models_F37['best.reduced_chi_square'] >= 10**0.5)/sum(np.logical_or(
    Models_F37['best.reduced_chi_square'] >= 10**0.5, Models_F37['best.reduced_chi_square'] <= 10**-0.5))
Cond12 = sum(Models_F37['best.reduced_chi_square'] <= 10**-0.5)/sum(np.logical_or(
    Models_F37['best.reduced_chi_square'] >= 10**0.5, Models_F37['best.reduced_chi_square'] <= 10**-0.5))
print('Percentage over and under fitting:', Cond11, Cond12)

Total number of galaxies: 8478
Percentage removed galaxies depending on condition: 0.7480111803913136 0.12040421414749516 0.22790797677918728
Percentage over and under fitting: 0.6501868352974993 0.34981316470250073


Here we count the classification types for the different CIGALE AGN models (To be used in Table 3). We read the VCV_TAP_otype.txt file to obtain the classifications and change the id in CIGALE models to a string format. Then, we join the tables and count the classifications.  

In [23]:
TV = Table.read('VCV_TAP_otype.txt', format='ascii').to_pandas()
SClean['main_id'] = np.array(SClean['id'], dtype='U30')
FClean['main_id'] = np.array(FClean['id'], dtype='U30')
SClean37['main_id'] = np.array(SClean37['id'], dtype='U30')
FClean37['main_id'] = np.array(FClean37['id'], dtype='U30')

In [24]:
SClass = SClean.to_pandas().join(TV.set_index('main_id'), on='main_id')
FClass = FClean.to_pandas().join(TV.set_index('main_id'), on='main_id')
S37Class = SClean37.to_pandas().join(TV.set_index('main_id'), on='main_id')
F37Class = FClean37.to_pandas().join(TV.set_index('main_id'), on='main_id')

##### SKIRTOR

In [26]:
print('Classified Seyfer galaxies:', sum(
    SClass.groupby('Sp').size()[['S1', 'S1n', 'S2', 'S']]))
print('Intermediate classifications', sum(SClass.groupby(
    'Sp').size()[['S1.0', 'S1.2', 'S1.5', 'S1.8', 'S1.9']]))
print('Other classifications', sum(SClass.groupby('Sp').size()[
      ['S1h', 'S1i', 'S1n0', 'S1n2', 'S1n5', 'S3', 'S3b', 'S?']]))
SClass.groupby('Sp').size()

Classified Seyfer galaxies: 7512
Intermediate classifications 511
Other classifications 71


Sp
S         29
S1      4576
S1.0     104
S1.2      94
S1.5     205
S1.8      66
S1.9      42
S1h       18
S1i        2
S1n     1410
S1n0       5
S1n2       3
S1n5       5
S2      1497
S3        25
S3b        1
S?        12
dtype: int64

In [66]:
SClass.groupby('otype_txt').size()

otype_txt
Sy1    6400
Sy2    1649
SyG      45
dtype: int64

##### Fritz

In [27]:
print('Classified Seyfer galaxies:', sum(
    FClass.groupby('Sp').size()[['S1', 'S1n', 'S2', 'S']]))
print('Intermediate classifications', sum(FClass.groupby(
    'Sp').size()[['S1.0', 'S1.2', 'S1.5', 'S1.8', 'S1.9']]))
print('Other classifications', sum(FClass.groupby('Sp').size()[
      ['S1h', 'S1i', 'S1n0', 'S1n2', 'S1n5', 'S3', 'S3b', 'S?']]))
FClass.groupby('Sp').size()

Classified Seyfer galaxies: 8026
Intermediate classifications 492
Other classifications 65


Sp
S         27
S1      4921
S1.0     101
S1.2      88
S1.5     195
S1.8      66
S1.9      42
S1h       12
S1i        1
S1n     1532
S1n0       4
S1n2       5
S1n5       5
S2      1546
S3        23
S3b        1
S?        14
dtype: int64

In [68]:
FClass.groupby('otype_txt').size()

otype_txt
Sy1    6834
Sy2    1706
SyG      43
dtype: int64

##### SKIRTOR with 30/70 angles

In [28]:
print('Classified Seyfer galaxies:', sum(
    S37Class.groupby('Sp').size()[['S1', 'S1n', 'S2', 'S']]))
print('Intermediate classifications', sum(S37Class.groupby(
    'Sp').size()[['S1.0', 'S1.2', 'S1.5', 'S1.8', 'S1.9']]))
print('Other classifications', sum(S37Class.groupby('Sp').size()[
      ['S1h', 'S1i', 'S1n0', 'S1n2', 'S1n5', 'S3', 'S3b', 'S?']]))
S37Class.groupby('Sp').size()

Classified Seyfer galaxies: 7580
Intermediate classifications 496
Other classifications 60


Sp
S         28
S1      4668
S1.0     100
S1.2      91
S1.5     198
S1.8      65
S1.9      42
S1h       14
S1i        2
S1n     1449
S1n0       3
S1n2       3
S1n5       4
S2      1435
S3        19
S3b        1
S?        14
dtype: int64

In [29]:
S37Class.groupby('otype_txt').size()

otype_txt
Sy1    6503
Sy2    1590
SyG      43
dtype: int64

##### Fritz with 30/70 angles

In [30]:
print('Classified Seyfer galaxies:', sum(
    F37Class.groupby('Sp').size()[['S1', 'S1n', 'S2', 'S']]))
print('Intermediate classifications', sum(F37Class.groupby(
    'Sp').size()[['S1.0', 'S1.2', 'S1.5', 'S1.8', 'S1.9']]))
print('Other classifications', sum(F37Class.groupby('Sp').size()
                                   [['S1h', 'S1i', 'S1n0', 'S1n2', 'S1n5', 'S3', 'S?']]))
F37Class.groupby('Sp').size()

Classified Seyfer galaxies: 7937
Intermediate classifications 482
Other classifications 59


Sp
S         28
S1      4974
S1.0      95
S1.2      85
S1.5     196
S1.8      64
S1.9      42
S1h       11
S1i        2
S1n     1533
S1n0       4
S1n2       5
S1n5       5
S2      1402
S3        19
S?        13
dtype: int64

In [76]:
F37Class.groupby('otype_txt').size()

otype_txt
Sy1    6875
Sy2    1561
SyG      42
dtype: int64

##### Notebook info

In [16]:
%load_ext watermark
%watermark -a "Andres Ramos" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy,pandas

Author: Andres Ramos

Python implementation: CPython
Python version       : 3.8.3
IPython version      : 7.16.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 3.10.0-1160.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 4.2
pandas : 1.2.0

numpy     : 1.19.5
autopep8  : 1.5.4
re        : 2.2.1
sys       : 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
logging   : 0.5.1.2
matplotlib: 3.2.2
json      : 2.0.9

Watermark: 2.1.0

